# 期末综合练习（2）

### 利用keras内置数据集练习文本数据分类（要求采用Conv1D + RNN模型）
路透社数据集：<BR>
路透社数据集包含许多短新闻及其对应的主题，由路透社在1986年发布。包括46个不同的主题，其中某些主题的样本更多，但是训练集中的每个主题都有至少10个样本。<BR>
与IMDB数据集一样，路透社数据集也内置到了Keras库中，并且已经经过了预处理。<BR>
#### 提示：
由于文本较长，先用CNN卷积上采样到较短长度，再用RNN处理是一个可以避免梯度消失的好方法。<BR>
    (由于卷积核为一维，卷积核大小要相应增大到5或7，stride增加到3或5)。
#### 要求：
利用callback将最佳模型保存到文件(注意：文件名应包含学号，保存在'save/'目录！)，
最后对最佳模型进行指标评估，展示混淆矩阵
#### 数据读取方法：
(x_train, y_train), (x_test, y_test) = keras.datasets.reuters.load_data(num_words=10000, test_split=0.2)

#### 考核办法：
score = model.evaluate(x_test, y_test)
计算得到的准确率为指标，准确率达到0.7为及格成绩起点，0.8以上优秀

In [1]:
#首先执行GPU资源分配代码，勿删除。
import tensorflow as tf
from tensorflow import keras
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.set_visible_devices(physical_devices[0], 'GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
max_features = 10000
maxlen = 10000

(x_train, y_train), (x_test, y_test) = keras.datasets.reuters.load_data(num_words=10000, test_split=0.2)
print(x_train.shape,x_test.shape)
print(len(x_train[0]), len(x_train[1]))

(8982,) (2246,)
87 56


In [3]:
word2index = keras.datasets.reuters.get_word_index()
index2word = dict([(value, key)for(key, value)in word2index.items()])
# 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
content =' '.join([index2word.get(i - 3, '?')for i in x_train[0]])
content

'? ? ? said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3'

In [4]:
import numpy as np
from keras.preprocessing import sequence

def vectorize_sequences(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results
x_train_vector = vectorize_sequences(x_train, maxlen) 
x_test_vector = vectorize_sequences(x_test, maxlen)
print(x_train_vector.shape)

x_train_pad = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test_pad = sequence.pad_sequences(x_test, maxlen=maxlen)
print(x_train_pad.shape)

print(x_train_vector[0])
print(x_train_pad[0])

(8982, 10000)
(8982, 10000)
[0. 1. 1. ... 0. 0. 0.]
[ 0  0  0 ... 15 17 12]


In [5]:
from keras.utils.np_utils import to_categorical
y_train_onehot = to_categorical(y_train)
y_test_onehot = to_categorical(y_test)
print(y_train_onehot.shape)

(8982, 46)


In [6]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

def model1():
    model = Sequential()
    model.add(Embedding(output_dim=32, input_dim=max_features, input_length=maxlen))
    model.add(Conv1D(32, 3, padding='same', activation='relu'))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(250, activation='relu'))
    model.add(Dense(46, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [7]:
model = model1()
save_path = 'save/3190105708_final2.h5'
checkpointer = ModelCheckpoint(filepath=save_path, monitor="val_accuracy", mode="max", verbose=1, save_best_only=True)
history = model.fit(x_train_vector, y_train_onehot, batch_size=1, epochs=5, verbose=1, validation_split=0.2, callbacks=[checkpointer])

Epoch 1/5


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[160000,250] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node RMSprop/RMSprop/update_3/mul_2 (defined at <ipython-input-7-417d42988ea8>:4) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[gradient_tape/sequential/embedding/embedding_lookup/Reshape/_20]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[160000,250] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node RMSprop/RMSprop/update_3/mul_2 (defined at <ipython-input-7-417d42988ea8>:4) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, 

#### 总结说明
此处说明关于模型设计与模型训练（参数设置、训练和调优过程）的心得与总结